In [ ]:



import re
import tweepy
import configparser
import numpy as np
from textblob import TextBlob
import statistics
from typing import List
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

# create the authentication object
auth = tweepy.OAuthHandler(api_key, api_key_secret)

#Set the access token and access token secret
auth.set_access_token(access_token, access_token_secret)

#Creat the API object while passing in the auth information
api = tweepy.API(auth)

# extract 100 tweets from twitter timeline


def get_tweets (keyword: str):
    all_tweets = []
    corsor = tweepy.Cursor(api.search, q=keyword , tweet_mode="extended",lang="ar"). items (3000)

    for tweet in corsor :
         all_tweets.append(tweet.full_text)
    return all_tweets


# clean the data
def clean_tweets (all_tweets: List[str]):
    tweets_clean = []
    for tweet in all_tweets:
      tweet = re.sub("@[A-Za-z0-9_]+","", tweet)
      tweet = re.sub("\n"," ", tweet)
      tweets_clean.append(tweet) # remove and clean the tweet from any "#" and hyper link

    tweets_clean= [tweet.replace(":","") for tweet in tweets_clean]
    return tweets_clean

# cleaned_tweets = clean_tweets(all_tweets)


def creatDataFrame(cleaned_tweets):
  #Create a dataframe with a column called Tweets
  df = pd.DataFrame( [tweet for tweet in cleaned_tweets] , columns=['Tweets'])
  return df

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
arb_stopwords = set(nltk.corpus.stopwords.words("arabic"))

def modify_arabic_post(text):
  text = re.sub('_+', ' ', text)
    ## To find arabic Tashkeel
  arabic_diacritics = re.compile("""
                              ّ    | # Tashdid
                              َ    | # Fatha
                              ً    | # Tanwin Fath
                              ُ    | # Damma
                              ٌ    | # Tanwin Damm
                              ِ    | # Kasra
                              ٍ    | # Tanwin Kasr
                              ْ    | # Sukun
                              ـ     # Tatwil/Kashida
                          """, re.VERBOSE)
  # Remove Tashkeel
  input_str = re.sub(arabic_diacritics, '', text)
  # Replace digits
  input_str = re.sub('[0-9]+', 'ارقام', input_str)
  s='٠١٢٣٤٥٦٧٨٩'
  input_str = re.sub('['+s+']+', 'ارقام', input_str)
  # Remove punctuation
  input_str = re.sub(r'[^\w\s]',' ',input_str)
  # Remove underscore
  input_str = re.sub('_+' , ' ', input_str)
  # Remove newLine
  input_str = re.sub(r'\n+', ' ', input_str)
  # Normalization
  input_str = re.sub("[إأآا]", "ا", input_str)
  input_str = re.sub("گ", "ك", input_str)
  # Remove duplicate ch more than 2
  pattern=re.compile(r"(.)\1{1,}",re.DOTALL)
  input_str = pattern.sub(r"\1\1",input_str)
  # Delete repeated words
  input_str = re.sub(r'\b(\w+)( \1\b)+', r'\1', input_str)
  # Delete non arabic characters
  regex = re.compile('[^0-9\u0621-\u064A\u0660-\u0669 ]')
  input_str = regex.sub('', input_str)
  # Delete consective spaces and spaces in begining and end
  input_str = input_str.strip()
  input_str = re.sub('\s+',' ',input_str)
  input_str = re.sub(' +', ' ', input_str)
  #stop word
  filtered_sentence = [w for w in input_str.split() if not w in arb_stopwords]
  input_str= ' '.join(filtered_sentence)
  #punctuation
  translator = str.maketrans('', '', punctuations_list)
  input_str= input_str.translate(translator)
  return input_str

In [ ]:
df['Tweets']=df['Tweets'].apply(modify_arabic_post)

In [ ]:
df

In [ ]:
def aa(input_str):
  input_str = re.sub("معصيتي راحتي", " ", input_str)
  input_str = re.sub("معصيت راحتي", " ", input_str)
  return input_str

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df['Tweets']=df['Tweets'].apply(aa)

In [ ]:
df

In [ ]:
df.to_csv('aa.csv',index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')